## Ejercicio 5 "Cadena Proteica" 

El objetivo de este ejercicio es a partir de un secuencia dada de DNA obtener los ORFs u open reading frames, para de esta forma traducirla a una proteina, asi como obtener la proteina mas larga
dentro de la secuencia input 

In [84]:
import Bio
import pandas as pd
from Bio.Seq import Seq 
from Bio import SeqUtils


def starts_stops(strand): 
    """
    Funcion que devuelve las posciones de inico y paro dentro de una cadena de DNA
    
    Parametros: 
    - strand (string): texto con la cadena de DNA a obtener sus posiciones de incio y paro 
    
    Returns: 
    - Lista(list)= lista cuyo primer indice son las posciones de inicio y el segundo las de paro 
    
    """
    Secuencia = Seq(strand)

    start_codon="ATG"

    #Posiciones de ORFs 
    pos= SeqUtils.nt_search(str(Secuencia), start_codon)[1:]

    # creamos una lista con los codones de paro 
    paro=["TAA", "TAG", "TGA"]

    #obtenemos toda las posciones de paro dentro de la secuencia
    pos_paro=[]
    for codon in paro: 
        pos_paro.append(SeqUtils.nt_search(str(Secuencia), codon)[1:])

    #Aplanamos la lista 
    pos_paro=[posicion for lista in pos_paro for posicion in lista]
    #ordenamos la lista 
    pos_paro.sort()

    lista=[pos, pos_paro]
    
    return lista


def Orfs(sec): 
    """
    Funcion que obtiene los open reading frames en los 3 marcos abiertos de lectura de una cadena 
    
    Parametros :
    sec (string): secuencia a obtener los ORFs
    
    Return: 
    Diccionario con los ORFs en cada marco de lectura 
    
    """
    #obtenemos la longitud de la secuencia
    longitud= len(sec) 
    #La estrategia es generara un array de datos con los indices de los codones en los marcos de lectura 
    marcos=[]
    for i in range(0,3): 
        marcos.append(list(range(i, longitud, 3))) 
        
    #obtenemos las posiciones con codones de incio / paro 
    posiciones=starts_stops(sec) 

    
    starts=posiciones[0]
    stops=posiciones[1]   
    
    Orfs={}
    for marco in marcos: 
        #obtenemos los ORFs en cada marco de acuerdo a las posiciones de los codone sde inicio y paro 
        pos_orfs=[] 
        starts_in_marco=[posicion for posicion in starts if posicion in marco]
        stops_in_marco=[posicion for posicion in stops if posicion in marco]
        for inicio in starts_in_marco: 
            for fin in stops_in_marco:
                if inicio < fin: 
                    pos_orfs.append([inicio,fin])
                    break 
                elif inicio > stops_in_marco[-1]:
                    pos_orfs.append([inicio,longitud])
                    break 
        Orfs[f"Orfs en el marco {marcos.index(marco)+1}"]={ 
                                f"Orf {pos_orfs.index(orf)}": [f"Inicia en la pos {orf[0]} y termina en la posicion {orf[1]}", sec[orf[0]:orf[1]+3]]
                                for orf in pos_orfs
                                } 
        
    return Orfs 



#Cadena de DNA del ejercicio  
ADN="AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAG" 
    
Reverse=ADN[::-1] 
print(Reverse)
#obtenemos los orfs en el marco del 1 al 3 
Orfs_fw=Orfs(ADN)
    
#Obtenemos los Orfs del 4 al 6 (strand reverse)
Orfs_rv=Orfs(Reverse) 

Orfs_fw

GACTCTACGATGAGCCTAGTAAGTCCGAATAAGGTTTTCTCTGAGATTAGGTTCAGCGCCCCAGTAGGGGTACATTGGACTCAATCGATGTACCGA


{'Orfs en el marco 1': {'Orf 0': ['Inicia en la pos 24 y termina en la posicion 66',
   'ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA'],
  'Orf 1': ['Inicia en la pos 30 y termina en la posicion 66',
   'ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA'],
  'Orf 2': ['Inicia en la pos 75 y termina en la posicion 96',
   'ATGATCCGAGTAGCATCTCAG']},
 'Orfs en el marco 2': {'Orf 0': ['Inicia en la pos 4 y termina en la posicion 7',
   'ATGTAG']},
 'Orfs en el marco 3': {}}

### Obtenemos la secuencia más larga 

In [88]:
# Obtenemos las longitudes de los distintos Orfs en un diccionario 
total_orfs={}
i=0
#recorremos los marcos de lectura en la cadena foward 
for dic in Orfs_fw.values(): 
    i+=1
    for key in dic.keys(): 
        total_orfs[f"Marco_{i}-{key}"]=[dic[key][1], len(dic[key][1])]

i=3
#recorremos los marcos de lectura en la cadena reverse
for dic in Orfs_rv.values(): 
    i+=1
    for key in dic.keys(): 
        total_orfs[f"Marco_{i}-{key}"]=[dic[key][1],len(dic[key][1])]    

#Creamos un data frame para manipular los datos 
orfs_df=pd.DataFrame.from_dict(total_orfs, orient="index")

#Renombramos las columnas
orfs_df.columns=["sec", "len"] 

#obtenemos la traduccion de la secuencia 
orfs_df["Prot"]=orfs_df["sec"].apply(lambda sec:Seq(sec).translate())

#Ordenamos las filas por su longitud de manera descendiente 
orfs_df.sort_values(by="len", axis=0, ascending=False)

#Optenemos la mas grande 
orf_grande=orfs_df.head(1) 


print(f"Los marcos de lectura encontrados son :\n {orfs_df}\n y el marco de lectura mas grande es:\n{orf_grande}")

Los marcos de lectura encontrados son :
                                                          sec  len  \
Marco_1-Orf 0  ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA   45   
Marco_1-Orf 1        ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA   39   
Marco_1-Orf 2                          ATGATCCGAGTAGCATCTCAG   21   
Marco_2-Orf 0                                         ATGTAG    6   

                                                        Prot  
Marco_1-Orf 0  (M, G, M, T, P, R, L, G, L, E, S, L, L, E, *)  
Marco_1-Orf 1        (M, T, P, R, L, G, L, E, S, L, L, E, *)  
Marco_1-Orf 2                          (M, I, R, V, A, S, Q)  
Marco_2-Orf 0                                         (M, *)  
 y el marco de lectura mas grande es:
                                                         sec  len  \
Marco_1-Orf 0  ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA   45   

                                                        Prot  
Marco_1-Orf 0  (M, G, M, T, P, R, L, G, L, E, S, L, L, E, 